# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

G. Perotti  ->  G. Perotti  |  ['G. Perotti']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
D. Christie  ->  D. Christie  |  ['D. Christie']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']


J. Li  ->  J. Li  |  ['J. Li']
T. Bertram  ->  T. Bertram  |  ['T. Bertram']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
M. Feldt  ->  M. Feldt  |  ['M. Feldt']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
Arxiv has 99 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.09627


extracting tarball to tmp_2407.09627...

 done.


G. Perotti  ->  G. Perotti  |  ['G. Perotti']


...

T. Bertram  ->  T. Bertram  |  ['T. Bertram']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
M. Feldt  ->  M. Feldt  |  ['M. Feldt']


Found 34 bibliographic references in tmp_2407.10588/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2407.10686


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{Brandl21}{Brandl}, B., {Bettonvil}, F., {van Boekel}, R., {Glauser}, A., {Quanz}, S.,  {Absil}, O., {Amorim}, A., {Feldt}, M., {Glasse}, A., {G{ü}del}, M., {Ho},  P., {Labadie}, L., {Meyer}, M., {Pantin}, E., {van Winckel}, H., and {METIS  Consortium}, ``{METIS: The Mid-infrared ELT Imager and Spectrograph},'' {\em  The Messenger}~{\bf 182},  22--26 (Mar. 2021).
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{Brandl24}{Brandl}, B., {Absil}, O., {Feldt}, M., {Garcia}, P., {Glasse}, A., {Guedel},  M., {Labadie}, L., {Meyer}, M., {Pantin}, E., {Quanz}, S., {Wang}, S

extracting tarball to tmp_2407.10686...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.09627-b31b1b.svg)](https://arxiv.org/abs/2407.09627) | **A JWST/MIRI analysis of the ice distribution and PAH emission in the protoplanetary disk HH 48 NE**  |
|| J. A. Sturm, et al. -- incl., <mark>G. Perotti</mark> |
|*Appeared on*| *2024-07-16*|
|*Comments*| *16 pages, 11 figures, accepted in A&A*|
|**Abstract**|            Ice-coated dust grains provide the main reservoir of volatiles that play an important role in planet formation processes and may become incorporated into planetary atmospheres. However, due to observational challenges, the ice abundance distribution in protoplanetary disks is not well constrained. We present JWST/MIRI observations of the edge-on disk HH 48 NE carried out as part of the IRS program Ice Age. We detect CO$_2$, NH$_3$, H$_2$O and tentatively CH$_4$ and NH$_4^+$. Radiative transfer models suggest that ice absorption features are produced predominantly in the 50-100 au region of the disk. The CO$_2$ feature at 15 micron probes a region closer to the midplane (z/r = 0.1-0.15) than the corresponding feature at 4.3 micron (z/r = 0.2-0.6), but all observations trace regions significantly above the midplane reservoirs where we expect the bulk of the ice mass to be located. Ices must reach a high scale height (z/r ~ 0.6; corresponding to modeled dust extinction Av ~ 0.1), in order to be consistent with the observed vertical distribution of the peak ice optical depths. The weakness of the CO$_2$ feature at 15 micron relative to the 4.3 micron feature and the red emission wing of the 4.3 micron CO$_2$ feature are both consistent with ices being located at high elevation in the disk. The retrieved NH$_3$ abundance and the upper limit on the CH$_3$OH abundance relative to H$_2$O are significantly lower than those in the interstellar medium (ISM), but consistent with cometary observations. Full wavelength coverage is required to properly study the abundance distribution of ices in disks. To explain the presence of ices at high disk altitudes, we propose two possible scenarios: a disk wind that entrains sufficient amounts of dust, thus blocking part of the stellar UV radiation, or vertical mixing that cycles enough ices into the upper disk layers to balance ice photodesorption.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.09799-b31b1b.svg)](https://arxiv.org/abs/2407.09799) | **The disc origin of the Milky Way bulge: On the high velocity dispersion of metal-rich stars at low latitude**  |
|| T. Boin, et al. -- incl., <mark>S. Ghosh</mark> |
|*Appeared on*| *2024-07-16*|
|*Comments*| *12 pages, 9 figures*|
|**Abstract**|            Previous studies of the chemo-kinematic properties of stars in the Galactic bulge have revealed a puzzling trend. Along the bulge minor axis, and close to the Galactic plane, metal-rich stars display a higher line-of-sight velocity dispersion compared to metal-poor stars, while at higher latitudes metal-rich stars have lower velocity dispersions than metal-poor stars, similar to what is found in the Galactic disc. In this work, we re-examine this issue, by studying the dependence of line-of-sight velocity dispersions on metallicity and latitude in the latest APOGEE Data Release 17, confirming the results of previous works. We then analyse an N-body simulation of a Milky Way-like galaxy, also taking into account observational biases introduced by the APOGEE selection function. We show that the inversion in the line-of-sight velocity dispersion-latitude relation observed in the Galactic bulge can be reproduced by our model. We show that this inversion is a natural consequence of a scenario in which the bulge is a boxy/peanut-shaped structure, whose metal-rich and metal-poor stars mainly originate from the thin and thick disc of the Milky Way, respectively. Due to their cold kinematics, metal-rich, thin disc stars, are efficiently trapped in the boxy/peanut bulge, and, at low latitudes, show a strong barred morphology, which results in high velocity dispersions which are larger than those attained by the metal-poor populations. Extremely metal-rich stars in the Galactic bulge, which have received renewed attention in the literature, do follow the same trends as those of the metal-rich populations. The line-of-sight velocity-latitude relation observed in the Galactic bulge for metal-poor and metal-rich stars are thus both an effect of the intrinsic nature of the Galactic bulge and of the angle at which we observe it from the Sun.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.10294-b31b1b.svg)](https://arxiv.org/abs/2407.10294) | **Inhomogeneous terminators on the exoplanet WASP-39 b**  |
|| N. Espinoza, et al. -- incl., <mark>D. Christie</mark>, <mark>X. Zhang</mark>, <mark>E.-M. Ahrer</mark>, <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-07-16*|
|*Comments*| *Published in Nature at this https URL. All code to produce plots (with data) can be found at this https URL*|
|**Abstract**|            Transmission spectroscopy has been a workhorse technique over the past two decades to constrain the physical and chemical properties of exoplanet atmospheres. One of its classical key assumptions is that the portion of the atmosphere it probes -- the terminator region -- is homogeneous. Several works in the past decade, however, have put this into question for highly irradiated, hot ($T_{eq}\gtrsim 1000$ K) gas giant exoplanets both empirically and via 3-dimensional modelling. While models predict clear differences between the evening (day-to-night) and morning (night-to-day) terminators, direct morning/evening transmission spectra in a wide wavelength range has not been reported for an exoplanet to date. Under the assumption of precise and accurate orbital parameters on WASP-39 b, here we report the detection of inhomogeneous terminators on the exoplanet WASP-39 b, which allows us to retrieve its morning and evening transmission spectra in the near-infrared ($2-5\ \mu$m) using JWST. We observe larger transit depths in the evening which are, on average, $405 \pm 88$ ppm larger than the morning ones, also having qualitatively larger features than the morning spectrum. The spectra are best explained by models in which the evening terminator is hotter than the morning terminator by $177^{+65}_{-57}$ K with both terminators having C/O ratios consistent with solar. General circulation models (GCMs) predict temperature differences broadly consistent with the above value and point towards a cloudy morning terminator and a clearer evening terminator.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.10588-b31b1b.svg)](https://arxiv.org/abs/2407.10588) | **METIS high-contrast imaging: from final design to manufacturing and testing**  |
|| O. Absil, et al. -- incl., <mark>T. Bertram</mark>, <mark>R. v. Boekel</mark>, <mark>M. Feldt</mark> |
|*Appeared on*| *2024-07-16*|
|*Comments*| *16 pages, paper presented at SPIE Astronomical Telescopes + Instrumentation 2024*|
|**Abstract**|            The Mid-infrared ELT Imager and Spectrograph (METIS) is one of the first-generation scientific instruments for the ELT, built under the supervision of ESO by a consortium of research institutes across and beyond Europe. Designed to cover the 3 to 13 $\mu$m wavelength range, METIS had its final design reviewed in Fall 2022, and has then entered in earnest its manufacture, assembly, integration, and test (MAIT) phase. Here, we present the final design of the METIS high-contrast imaging (HCI) modes. We detail the implementation of the two main coronagraphic solutions selected for METIS, namely the vortex coronagraph and the apodizing phase plate, including their combination with the high-resolution integral field spectrograph of METIS, and briefly describe their respective backup plans (Lyot coronagraph and shaped pupil plate). We then describe the status of the MAIT phase for HCI modes, including a review of the final design of individual components such as the vortex phase masks, the grayscale ring apodizer, and the apodizing phase plates, as well as a description of their on-going performance tests and of our plans for system-level integration and tests. Using end-to-end simulations, we predict the performance that will be reached on sky by the METIS HCI modes in presence of various environmental and instrumental disturbances, including non-common path aberrations and water vapor seeing, and discuss our strategy to mitigate these various effects. We finally illustrate with mock observations and data processing that METIS should be capable of directly imaging temperate rocky planets around the nearest stars.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.09949-b31b1b.svg)](https://arxiv.org/abs/2407.09949) | **The formation of supermassive black holes from Population III.1 seeds. III. Galaxy evolution and black hole growth from semi-analytic modelling**  |
|| V. Cammelli, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2024-07-16*|
|*Comments*| *Submitted to MNRAS, comments welcome*|
|**Abstract**|            We present an implementation of Pop III.1 seeding of supermassive black holes (SMBHs) in a theoretical model of galaxy formation and evolution to assess the growth the SMBH population and the properties of the host galaxies. The model of Pop III.1 seeding involves SMBH formation at redshifts $z\gtrsim 20$ in dark matter minihalos that are isolated from external radiative feedback, parameterized by isolation distance $d_{\rm iso}$. Within a standard $\Lambda$CDM cosmology, we generate dark matter halos using the code \textsc{pinocchio} and seed them according to the Pop III.1 scenario, exploring values of $d_{\rm iso}$ from 50 to 100~kpc (proper distance). We consider two alternative cases of SMBH seeding: a Halo Mass Threshold (HMT) model in which all halos $>7\times10^{10}\:M_\odot$ are seeded with $\sim 10^5\:M_\odot$ black holes; an All Light Seed (ALS) model in which all halos are seeded with low, stellar-mass black holes. We follow the redshift evolution of the halos, populating them with galaxies using the GAlaxy Evolution and Assembly theoretical model of galaxy formation, including accretion on SMBHs and related feedback processes. Here we present predictions for the properties of galaxy populations, focusing on stellar masses, star formation rates, and black hole masses. The local, $z\sim0$ metrics of occupation fraction as a function of the galaxy stellar mass, galaxy stellar mass function (GSMF), and black hole mass function (BHMF) all suggest a constraint of $d_{\rm iso}<75\:$kpc. We discuss the implications of this result for the Pop III.1 seeding mechanism.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.10540-b31b1b.svg)](https://arxiv.org/abs/2407.10540) | **Sudden polarization angle jumps of the repeating fast radio burst FRB 20201124A**  |
|| J. R. Niu, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-07-16*|
|*Comments*| *10 pages, 5 figures, submitted to APJL*|
|**Abstract**|            We report the first detection of polarization angle (PA) orthogonal jumps, a phenomenon previously only observed from radio pulsars, from a fast radio burst (FRB) source FRB 20201124A. We find three cases of orthogonal jumps in over two thousand bursts, all resembling those observed in pulsar single pulses. We propose that the jumps are due to the superposition of two orthogonal emission modes that could only be produced in a highly magnetized plasma, and they are caused by the line of sight sweeping across a rotating magnetosphere. The shortest jump timescale is of the order of one-millisecond, which hints that the emission modes come from regions smaller than the light cylinder of most pulsars or magnetars. This discovery provides convincing evidence that FRB emission originates from the complex magnetosphere of a magnetar, suggesting an FRB emission mechanism that is analogous to radio pulsars despite a huge luminosity difference between two types of objects.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.10686-b31b1b.svg)](https://arxiv.org/abs/2407.10686) | **Beyond the horizon: Quantifying the full sky foreground wedge in the cylindrical power spectrum**  |
|| S. Munshi, et al. -- incl., <mark>S. Ghosh</mark> |
|*Appeared on*| *2024-07-16*|
|*Comments*| *21 pages, 13 figures, and 2 tables; submitted to Astronomy and Astrophysics (A&A)*|
|**Abstract**|            One of the main obstacles preventing the detection of the redshifted 21-cm signal from neutral hydrogen in the early Universe is the astrophysical foreground emission, which is several orders of magnitude brighter than the signal. The foregrounds, due to their smooth spectra, are expected to predominantly occupy a region in the cylindrical power spectrum known as the foreground wedge. However, the conventional equations describing the extent of the foreground wedge are derived under a flat-sky approximation. This assumption breaks down for tracking wide-field instruments, thus rendering these equations inapplicable in these situations. In this paper, we derive equations for the full sky foreground wedge and show that the foregrounds can potentially extend far beyond what the conventional equations suggest. We also derive the equations that describe a specific bright source in the cylindrical power spectrum space. The validity of both sets of equations is tested against numerical simulations. Many current and upcoming interferometers (e.g., LOFAR, NenuFAR, MWA, SKA) are wide-field phase-tracking instruments. These equations give us new insights into the nature of foreground contamination in the cylindrical power spectra estimated using wide-field instruments. Additionally, they allow us to accurately associate features in the power spectrum to foregrounds or instrumental effects. The equations are also important for correctly selecting the "EoR window" for foreground avoidance analyses, and for planning 21-cm observations. In future analyses, it is recommended to use these updated horizon lines to indicate the foreground wedge in the cylindrical power spectrum accurately. The new equations for generating the updated wedge lines are made available in a Python library, pslines.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.09810-b31b1b.svg)](https://arxiv.org/abs/2407.09810) | **Euclid and KiDS-1000: Quantifying the impact of source-lens clustering on cosmic shear analyses**  |
|| L. Linke, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-07-16*|
|*Comments*| *17 pages plus appendix, 10 figures, abstract abridged for arXiv*|
|**Abstract**|            Cosmic shear is a powerful probe of cosmological models and the transition from current Stage-III surveys like the Kilo-Degree Survey (KiDS) to the increased area and redshift range of Stage IV-surveys such as \Euclid will significantly increase the precision of weak lensing analyses. However, with increasing precision, the accuracy of model assumptions needs to be evaluated. In this study, we quantify the impact of the correlated clustering of weak lensing source galaxies with the surrounding large-scale structure, the so-called source-lens clustering (SLC), which is commonly neglected. We include the impact of realistic scatter in photometric redshift estimates, which impacts the assignment of galaxies to tomographic bins and increases the SLC. For this, we use simulated cosmological datasets with realistically distributed galaxies and measure shear correlation functions for both clustered and uniformly distributed source galaxies. Cosmological analyses are performed for both scenarios to quantify the impact of SLC on parameter inference for a KiDS-like and a \Euclid-like setting. We find for Stage III surveys like KiDS, SLC has a minor impact when accounting for nuisance parameters for intrinsic alignments and shifts of tomographic bins, as these nuisance parameters absorb the effect of SLC, thus changing their original meaning. For KiDS (\Euclid), the inferred intrinsic alignment amplitude $A_\mathrm{IA}$ changes from $0.11_{-0.46}^{+0.44}$ ($-0.009_{-0.080}^{+0.079}$) for data without SLC to $0.28_{-0.44}^{+0.42}$ ($0.022_{-0.082}^{+0.081}$) with SLC. However, fixed nuisance parameters lead to shifts in $S_8$ and $\Omega_\mathrm{m}$. For \Euclid we find that $S_8$ and $\Omega_\mathrm{m}$ are shifted by 0.14 and 0.12 $\sigma$, respectively, when including free nuisance parameters. Consequently, SLC on its own has only a small impact on the inferred parameters.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.09627/./images/CBF_paper.png', 'tmp_2407.09627/./images/sed_modelcomparison.png', 'tmp_2407.09627/./images/sketch_turbulence_wind.png']
copying  tmp_2407.09627/./images/CBF_paper.png to _build/html/
copying  tmp_2407.09627/./images/sed_modelcomparison.png to _build/html/
copying  tmp_2407.09627/./images/sketch_turbulence_wind.png to _build/html/
exported in  _build/html/2407.09627.md
    + _build/html/tmp_2407.09627/./images/CBF_paper.png
    + _build/html/tmp_2407.09627/./images/sed_modelcomparison.png
    + _build/html/tmp_2407.09627/./images/sketch_turbulence_wind.png
found figures ['tmp_2407.09799/./figures/xy_vx.png', 'tmp_2407.09799/./figures/xy_vy.png', 'tmp_2407.09799/./figures/dispersion_comparison_30_1_5.png', 'tmp_2407.09799/./figures/dispersion_comparison.png', 'tmp_2407.09799/./figures/xy_lb.png']
copying  tmp_2407.09799/./figures/xy_vx.png to _build/html/
copying  tmp_2407.09799/./figures/xy_vy.png to _build/html/
copying  tmp_2407.09799/./figures

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\micron}{\textmu m\xspace}$
$\newcommand{\ditto}[1][.0pt]{\xrfill[.7ex]{#1}~\textquotedbl~\xrfill[.7ex]{#1}}$
$\newcommand{\as}[1]{{\color{ForestGreen}AS: #1}}$
$\newcommand{\jn}[1]{{\color{red!50!blue}\;[JN:~#1]}}$
$\newcommand{\ed}[1]{\textcolor{darkorange}{[ED:~#1]}}$
$\newcommand{\SubItem}[1]{{\setlength\itemindent{15pt}\item[*]#1}}$
$\newcommand{\orcidlink}[1]{\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt]{orcid.png}}}$</div>



<div id="title">

# A JWST/MIRI analysis of the ice distribution and PAH emission in the protoplanetary disk HH 48 NE

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.09627-b31b1b.svg)](https://arxiv.org/abs/2407.09627)<mark>Appeared on: 2024-07-16</mark> -  _16 pages, 11 figures, accepted in A&A_

</div>
<div id="authors">

J. A. Sturm, et al. -- incl., <mark>G. Perotti</mark>

</div>
<div id="abstract">

**Abstract:** Ice-coated dust grains provide the main reservoir of volatiles that play an important role in planet formation processes and may become incorporated into planetary atmospheres.    However, due to observational challenges, the ice abundance distribution in protoplanetary disks is not well constrained.    With the advent of the _James Webb_ Space Telescope (JWST) we are in a unique position to observe these ices in the near- to mid-infrared and constrain their properties in Class II protoplanetary disks. We present JWST Mid-InfraRed Imager (MIRI) observations of the edge-on disk HH 48 NE carried out as part of the Director’s Discretionary Early Release Science program Ice Age, completing the ice inventory of HH 48 NE by combining the MIRI data (5 -- 28 $\micron$ ) with those of NIRSpec (2.7 -- 5 $\micron$ ). We used radiative transfer models tailored to the system, including silicates, ices, and polycyclic aromatic hydrocarbons (PAHs) to reproduce the observed spectrum of HH 48 NE with a parameterized model.    The model was then used to identify ice species and constrain spatial information about the ices in the disk. The mid-infrared spectrum of HH 48 NE is relatively flat with weak ice absorption features.    We detect $\ce{CO2}$ , $\ce{NH3}$ , $\ce{H2O}$ and tentatively $\ce{CH4}$ and $\ce{NH4+}$ .    Radiative transfer models suggest that ice absorption features are produced predominantly in the 50 -- 100 au region of the disk. The $\ce{CO2}$ feature at 15 $\micron$ probes a region closer to the midplane ( $z/r$ = 0.1 -- 0.15) than the corresponding feature at 4.3 $\micron$ ( $z/r$ = 0.2 -- 0.6), but all observations trace regions significantly above the midplane reservoirs where we expect the bulk of the ice mass to be located.    Ices must reach a high scale height ( $z/r \sim 0.6$ ; corresponding to modeled dust extinction $A_{\rm v} \sim 0.1$ ), in order to be consistent with the observed vertical distribution of the peak ice optical depths.    The weakness of the $\ce{CO2}$ feature at 15 $\micron$ relative to the 4.3 $\micron$ feature and the red emission wing of the 4.3 $\micron$ $\ce{CO2}$ feature are both consistent with ices being located at high elevation in the disk.    The retrieved $\ce{NH3}$ abundance and the upper limit on the $\ce{CH3OH}$ abundance relative to $\ce{H2O}$ are significantly lower than those in the interstellar medium (ISM), but consistent with cometary observations.    The contrast of the PAH emission features with the continuum is stronger than for similar face-on protoplanetary disks, which is likely a result of the edge-on system geometry.    Modeling based on the relative strength of the emission features suggests that the PAH emission originates in the disk surface layer rather than the ice absorbing layer. Full wavelength coverage is required to properly study the abundance distribution of ices in disks.    To explain the presence of ices at high disk altitudes, we propose two possible scenarios: a disk wind that entrains sufficient amounts of dust, thus blocking part of the stellar UV radiation, or vertical mixing that cycles enough ices into the upper disk layers to balance ice photodesorption from the grains.

</div>

<div id="div_fig1">

<img src="tmp_2407.09627/./images/CBF_paper.png" alt="Fig10" width="100%"/>

**Figure 10. -** Top left: total dust density distribution in the model with internal shielding (radial $A_\mathrm{V}$; black dashed) at 0.03, 0.1, 0.3, 1, and 10 mag and external shielding (vertical $A_\mathrm{V}$; white dashed) at 0.01, 0.03, 0.1, and 1 mag from top to bottom. The other panels show the contribution function (CBF), i.e., the contribution of a specific region to the total absorption of the ice feature in percentages. The pure CO ice at high abundance is added for reference, but we argue in the main text that this component is not observed. The absorbing area is the region in the disk that contribute significantly to the total absorption. The red and yellow lines indicate the fiducial and best fit photodesorption snow surface, respectively, for reference. The thick, dashed, white line shows the $\tau=1$ surface at the ice feature wavelength. We would like to note that viewing angle is really important and is 83◦ in this case (*fig:CBF*)

</div>
<div id="div_fig2">

<img src="tmp_2407.09627/./images/sed_modelcomparison.png" alt="Fig8" width="100%"/>

**Figure 8. -** Overview of the source integrated spectrum of HH 48 NE (grey), with the continuum baseline underneath the gas lines shown in black with a zoom-in on the JWST wavelength range 2.7 -- 27 $\micron$. Different ice radiative transfer models are shown for comparison with the observations. The initial model constrained from photometric data points, HST, and ALMA observations  ([Sturm, McClure and Law 2023](), [Sturm, McClure and Bergner 2023]())  is shown in blue. A similar model was fine-tuned to the NIRSpec and MIRI spectrum with only refractory dust (red), dust and ice (green), and dust, ice, and PAHs (orange). The PAH opacity template used is shown in the bottom without extinction corrections  ([Visser, Geers and Dullemond 2007]()) .
     (*fig:SED comparison*)

</div>
<div id="div_fig3">

<img src="tmp_2407.09627/./images/sketch_turbulence_wind.png" alt="Fig12" width="100%"/>

**Figure 12. -** Sketch of the three pieces of evidence for elevated ices in HH 48 NE. The orange line demonstrates the effect of including the ices high up in the disk on the vertical absorption profile (panel 1.), the relative strength of the 4.3 to 15 $\micron$\ce{CO2} features (panel 2.), and the scattering wing of the 4.3 $\micron$\ce{CO2} feature (panel 3.). Two scenarios are proposed: a dusty disk wind blocking part of the UV emission from the star (Scenario A.) or vertical mixing stirring up ice to the surface (Scenario B.). (*fig:sketch*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.09627"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\paola}[1]{\textcolor{magenta}{{#1}}}$</div>



<div id="title">

# The disc origin of the Milky Way bulge

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.09799-b31b1b.svg)](https://arxiv.org/abs/2407.09799)<mark>Appeared on: 2024-07-16</mark> -  _12 pages, 9 figures_

</div>
<div id="authors">

T. Boin, et al. -- incl., <mark>S. Ghosh</mark>

</div>
<div id="abstract">

**Abstract:** Previous studies of the chemo-kinematic properties of stars in the Galactic bulge have revealed a puzzling trend. Along the bulge minor axis,  and close to the Galactic plane, metal-rich stars display a higher line-of-sight velocity dispersion compared to metal-poor stars, while at higher latitudes metal-rich stars have lower velocity dispersions than metal-poor stars, similar to what is found in the Galactic disc. In this work, we re-examine this issue, by studying the dependence of line-of-sight velocity dispersions on metallicity and latitude in the latest APOGEE Data Release 17, confirming the results of previous works. We then analyse an $N$ -body simulation of a Milky Way-like galaxy, also taking into account observational biases introduced by the APOGEE selection function. We show that the inversion in the line-of-sight velocity dispersion-latitude relation observed in the Galactic bulge -- where the velocity dispersion of metal-rich stars becomes greater than that of metal-poor stars as latitude decreases -- can be reproduced by our model.  We show that this inversion is a natural consequence of a scenario in which the bulge is  a boxy/peanut-shaped structure, whose metal-rich and metal-poor  stars mainly originate from the thin and thick disc of the Milky Way, respectively. Due to their cold kinematics, metal-rich, thin disc stars, are efficiently trapped in the boxy/peanut bulge, and, at low latitudes, show a strong barred morphology, which - given the bar orientation with respect to the Sun-Galactic centre direction  - results in high velocity dispersions which are larger than those attained by the metal-poor populations. Extremely metal-rich stars in the Galactic bulge, which have received renewed attention in the literature, do follow the same trends as those of the metal-rich populations. The  line-of-sight velocity-latitude relation observed in the Galactic bulge for metal-poor and metal-rich stars are thus both an effect of the intrinsic nature of the Galactic bulge (i.e. mostly secular) and of the angle at which we observe it from the Sun.

</div>

<div id="div_fig1">

<img src="tmp_2407.09799/./figures/xy_vx.png" alt="Fig4.1" width="50%"/><img src="tmp_2407.09799/./figures/xy_vy.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Maps of the velocity dispersion maps for the bar-oriented $\sigma_{v_{Xb}}$(_left panel_) and $\sigma_{v_{Yb}}$(_right panel_) components. Each population (metal-rich, metal-intermediate and metal-poor) correspond to a column in each panel, and different height cuts are represented on each row (0 kpc < |Z| < 0.25 kpc, 0.25 kpc < |Z| < 0.5 kpc and 0.5 kpc < |Z| < 1 kpc). The axis are bar-oriented, isodensity contours are shown as colored lines, and a 1.5 kpc region around the galactic centre is shown as a dotted black circle. (*fig:xy_v*)

</div>
<div id="div_fig2">

<img src="tmp_2407.09799/./figures/dispersion_comparison_30_1_5.png" alt="Fig3.1" width="50%"/><img src="tmp_2407.09799/./figures/dispersion_comparison.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** _Left panel_: Line-of-sight velocity dispersion $\sigma_{v_{los}}$ profiles of our quality APOGEE sample (_error bar curves_) and of our mock catalogue (_shaded areas of height $\pm \sigma$_). The profiles are plotted for stars with |$\ell$|$<2^\circ$ and |X| $<$ 1.5kpc. _Right panel_: Line-of-sight velocity dispersion $\sigma_{v_{los}}$ profiles of our quality APOGEE sample (_error bar with circle symbol_) and of our mock catalogue (_shaded areas of height $\pm \sigma$_), compared to results from [ and Babusiaux (2014)]() and [ and Babusiaux (2010)]()(_squares_), Ness_2013(_pentagons_), [ and Rojas-Arriagada (2014)]()(_triangles_), [ and Uttenthaler (2012)]()(_diamonds_), [ and Zoccali (2017)]()(_stars_). (*fig:disp_apogee_sim*)

</div>
<div id="div_fig3">

<img src="tmp_2407.09799/./figures/xy_lb.png" alt="Fig2" width="100%"/>

**Figure 2. -** _Top panel:_ face-on (X-Y-plane) density map of the $N$-Body simulation stars _(left column)_, APOGEE stars _(middle column)_ and our mock catalogue stars _(right column)_. Superimposed are the |X| < 1.5kpc distance bounds and $\pm$ 20  longitude bounds in white, as well as a green line to guide the eye indicating the angle of the galactic bar and the Sun  at X=-8.12 kpc,Y=0 kpc (white star symbol). _Bottom panels:_ corresponding density maps in the Galactic ($\ell$,$b$) plane. The colors indicate relative densities for each of the 3 catalogues separately. (*fig:xy_lb*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.09799"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\doi}[1]{\url{https://doi.org/#1}}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\doi}[1]{\url{https://doi.org/#1}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\floatpagefraction}{1}$
$\newcommand{\floatpagefraction}{1}$
$\newcommand{\figurename}{Extended Data Fig.}$
$\newcommand{\tablename}{Extended Data Table}$
$\newcommand{\theHfigure}{Extended Data Fig.~\arabic{figure}}$
$\newcommand{\thetable}{Extended Data Table \arabic{table}}$
$\newcommand{\theHtable}{Extended Data Table \arabic{table}}$
$\newcommand\url{#1}$
$\newcommand{\urlprefix}{URL }$
$\newcommand\url{#1}$
$\newcommand{\urlprefix}{URL }$</div>



<div id="title">

# $\centering{Inhomogeneous terminators on the exoplanet WASP-39 b}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.10294-b31b1b.svg)](https://arxiv.org/abs/2407.10294)<mark>Appeared on: 2024-07-16</mark> -  _Published in Nature at this https URL. All code to produce plots (with data) can be found at this https URL_

</div>
<div id="authors">

\fnm{Néstor}\sur{Espinoza}, et al.

</div>
<div id="abstract">

**Abstract:**            Transmission spectroscopy has been a workhorse technique over the past two decades to constrain the physical and chemical properties of exoplanet atmospheres. One of its classical key assumptions is that the portion of the atmosphere it probes -- the terminator region -- is homogeneous. Several works in the past decade, however, have put this into question for highly irradiated, hot ($T_{eq}\gtrsim 1000$ K) gas giant exoplanets both empirically and via 3-dimensional modelling. While models predict clear differences between the evening (day-to-night) and morning (night-to-day) terminators, direct morning/evening transmission spectra in a wide wavelength range has not been reported for an exoplanet to date. Under the assumption of precise and accurate orbital parameters on WASP-39 b, here we report the detection of inhomogeneous terminators on the exoplanet WASP-39 b, which allows us to retrieve its morning and evening transmission spectra in the near-infrared ($2-5\ \mu$m) using JWST. We observe larger transit depths in the evening which are, on average, $405 \pm 88$ ppm larger than the morning ones, also having qualitatively larger features than the morning spectrum. The spectra are best explained by models in which the evening terminator is hotter than the morning terminator by $177^{+65}_{-57}$ K with both terminators having C/O ratios consistent with solar. General circulation models (GCMs) predict temperature differences broadly consistent with the above value and point towards a cloudy morning terminator and a clearer evening terminator.         

</div>

<div id="div_fig1">

<img src="tmp_2407.10294/./model_plus_data.*.png" alt="Fig2" width="100%"/>

**Figure 2. -** **The morning and evening spectrum of WASP-39 b from JWST NIRSpec/PRISM observations.****a-b.** The total transit depth by adding the morning and evening spectra (**a**, grey points) along with the individual morning (blue points) and evening (red points) spectrum of WASP-39 b as derived from our lightcurve modeling (**b**). Big points are datapoints at $R=30$ shown for illustration; smaller points are at $R=100$. The best-fit, chemically-consistent models (black solid line in **a**, red and blue solid lines in **b**; fitted to the $R=100$ spectra) are consistent with a hotter evening terminator (see text for details). **c-d.** Residuals from the best-fit model (red points for the evening, blue points for the morning); dashed line marks 0. {All errorbars represent 1-standard deviation.} (*fig:morning-evening-spectra*)

</div>
<div id="div_fig2">

<img src="tmp_2407.10294/./limb_experiment_diff.png" alt="Fig7.1" width="50%"/><img src="tmp_2407.10294/./limb_all_exps.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** **Robustness of limb asymmetry detection to transit parameters and assumptions.** To study the robustness of our extracted morning and evening spectra for the NIRSpec/PRISM
    observations, we simulated transit lightcurves using
    \texttt{batman} and then fitted those using \texttt{catwoman} with different assumptions, leaving all parameters fixed but the depths of
    the morning and evening limbs. **a-b.** Null case on which the
    true model has no limb asymmetries and the input transit parameters are unchanged; the \texttt{catwoman} fits
    correctly recover the same morning (blue) and evening (red) spectra
    (top). The difference $\Delta$ between the morning and the evening
    spectra are consistent with zero, as expected from this null case (bottom). **c.** Same experiment
    as in a., but generating a light curve with a time-of-transit center 3-$\sigma$ away from the fixed value, which amounts to an offset of 3.4
    seconds. The difference $\Delta$ is consistent with zero, suggesting
    our inferences are robust against this parameter. **d.** Same
    experiment, but generating a light curve that had limb-darkening
    coefficients of the quadratic law offset by 0.01. Note how this injects
    a systematic offset in the difference between the morning and evening
    spectra; depending on the direction of this offset, this can lead to
    mornings having larger depths than evenings or viceversa. **e.**
    Same experiment, but generating a transit lightcurve with a non-zero
    eccentricity consistent at 3-$\sigma$ with the white-lightcurve fits of Carter \& May et al. (in review; $e=0.035$, $\omega = 10$ deg). Note how this slight eccentricity can
    generate significantly larger mornings than evenings, due to the asymmetry an eccentric orbit imprints on the transit lightcurve. For WASP-39 b, this eccentricity effect cannot generate larger evenings than mornings, which is what we observe. This suggests our results are also robust against this parameter (see text for details). {All errorbars represent 1-standard deviation.} (*fig:experiments*)

</div>
<div id="div_fig3">

<img src="tmp_2407.10294/./data.png" alt="Fig4" width="100%"/>

**Figure 4. -** **Different approaches at detecting limb asymmetries from NIRSpec/PRISM data.****a-b.** Evening (top) and morning (middle) depths as extracted from three independent analyses of our NIRSpec/PRISM lightcurves; one using the \texttt{catwoman} framework with limb-darkening as free parameters with a prior (NE), a framework leaving those fixed (MM) and a framework on which half-ingress and half-egress are fitted independently using a \texttt{batman} lightcurve model (Tiberius); see text for details. Note the agreement between approaches for both terminators, and how the amplitude of the features seem to be smaller in the morning terminator **c.** An independent look at limb asymmetries by fitting for a wavelength-dependent time-of-transit center to each wavelength-dependent lightcurve. As with the top and middle panels, differences between the limbs as tracked by the time-of-transit center seem to be largest between
    2-3.5 $\mu$m, i.e., around the water bands. {All errorbars represent 1-standard deviation.} (*fig:all-data*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.10294"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\baselinestretch}{1.0}$</div>



<div id="title">

# METIS high-contrast imaging: from final design to manufacturing and testing

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.10588-b31b1b.svg)](https://arxiv.org/abs/2407.10588)<mark>Appeared on: 2024-07-16</mark> -  _16 pages, paper presented at SPIE Astronomical Telescopes + Instrumentation 2024_

</div>
<div id="authors">

O. Absil, et al. -- incl., <mark>T. Bertram</mark>, <mark>R. v. Boekel</mark>, <mark>M. Feldt</mark>

</div>
<div id="abstract">

**Abstract:** The Mid-infrared ELT Imager and Spectrograph (METIS) is one of the first-generation scientific instruments for the ELT, built under the supervision of ESO by a consortium of research institutes across and beyond Europe. Designed to cover the 3 to 13 $\mu$ m wavelength range, METIS had its final design reviewed in Fall 2022, and has then entered in earnest its manufacture, assembly, integration, and test (MAIT) phase. Here, we present the final design of the METIS high-contrast imaging (HCI) modes. We detail the implementation of the two main coronagraphic solutions selected for METIS, namely the vortex coronagraph and the apodizing phase plate, including their combination with the high-resolution integral field spectrograph of METIS, and briefly describe their respective backup plans (Lyot coronagraph and shaped pupil plate). We then describe the status of the MAIT phase for HCI modes, including a review of the final design of individual components such as the vortex phase masks, the grayscale ring apodizer, and the apodizing phase plates, as well as a description of their on-going performance tests and of our plans for system-level integration and tests. Using end-to-end simulations, we predict the performance that will be reached on sky by the METIS HCI modes in presence of various environmental and instrumental disturbances, including non-common path aberrations and water vapor seeing, and discuss our strategy to mitigate these various effects. We finally illustrate with mock observations and data processing that METIS should be capable of directly imaging temperate rocky planets around the nearest stars.

</div>

<div id="div_fig1">

<img src="tmp_2407.10588/./pupil_stops_RAP.png" alt="Fig2" width="100%"/>

**Figure 2. -** lyot_Left._ ELT input pupil, illustrating our dodecagonal/hexagonal all-glass approximation (white) compared to the full input pupil (gray). _Middle._ Design of the Lyot stops associated with the classical (middle left) and ring-apodized (middle right) vortex modes, as implemented at IMG-LM-PP1. Sizes are in given in mm, with the ELT nominal diameter corresponding to 45 mm in this pupil plane. _Right._ Design of the ring apodizer (RAP). Sizes are in given in mm, with the ELT nominal diameter corresponding to 68 mm in this pupil plane. The relative dimensions of all masks with respect to the input ELT pupil are respected to illustrate the amount of oversizing or undersizing in the various edges. (*fig:lyot*)

</div>
<div id="div_fig2">

<img src="tmp_2407.10588/./APP-IMG.png" alt="Fig3" width="100%"/>

**Figure 3. -** APP-IMG phase pattern and resultant monochromatic PSF. _Left._ Phase of the vAPP as a function of position in the METIS cold stop pupil. _Right._ The two (monochromatic) coronagraphic PSFs formed by this phase pattern, located on either side of a central leakage term of about 1\% intensity relative to the input on-axis starlight. (*fig:app-img*)

</div>
<div id="div_fig3">

<img src="tmp_2407.10588/./VPM-N4.png" alt="Fig5.1" width="50%"/><img src="tmp_2407.10588/./VPM-N8.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Measured performance of two of the three N-band VPMs manufactured for METIS (_left_: VPM-N4, _right_: VPM-N8). Not displayed here is VPM-N9, which shows very similar performance to VPM-N8. The final, consolidated measurements are displayed in orange, with ealier results in shades of gray. In the case of VPM-N8, the gray-colored measurements were obtained before the re-etching of the VPM, showing poor early performance. The requirement in terms of average rejection rate across the N band is displayed with a green dashed line for reference. (*fig:vpm-n*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.10588"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

137  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

20  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
